# PA3b Sentiment Classification
#### Applied Machine Learning
Grpup 39: Sebastian Kölbel & Min Ze Teh

We begin by importing libraries needed for the task

In [180]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# for converting training and test datasets into matrices
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB, BernoulliNB

We create a function for preprocessing the csv-files. These preprocessing steps include:
* Making text all lower case
* Removing leading and closing white-space
* Make sure punctuations are separated from words

In [142]:
def read_sentiments(doc_file):
    with open(doc_file, 'r', encoding='utf-8') as f:
        new_lines = []
        for line in f:
            line = line.lower().removesuffix('\n').strip().replace()
            new_lines.append(line.split('\t'))
                
        return pd.DataFrame(new_lines,columns=['sentiment','text'])

We then separate the input values from the output values. The crowdsourced data needed some more preprocessing. There we too many values for the sentiments all consisting of typos. We checked how many errors there were, however there were only 75 errors out of more than 10 000 rows so the errors were dropped to save time.

In [25]:
df = read_sentiments('Data/crowdsourced_train.csv').drop(0)
cs_training = df[df['sentiment'].isin(['positive','negative','neutral'])]
print('Dropped rows in crowdsourced data:',len(df)-len(cs_training))
X_cs_train = cs_training.drop('sentiment', axis=1)
Y_cs_train = cs_training['sentiment']

gold_training = read_sentiments('Data/gold_train.csv').drop(0)
X_gold_train = gold_training.drop('sentiment', axis=1)
Y_gold_train = gold_training['sentiment']


testing = read_sentiments('Data/test.csv').drop(0)
X_test = testing.drop('sentiment', axis=1)
Y_test = testing['sentiment']

Dropped rows in crowdsourced data: 75


In [181]:
def train_document_classifier(X, Y):
    pipeline = make_pipeline(
        # punctuations are removed here, max df should make sure that no too common words are present
        TfidfVectorizer(encoding='utf-8',  
                        max_df=0.05), 
        BernoulliNB())
    pipeline.fit(X, Y)
    return pipeline

In [202]:
def train_document_classifier2(X, Y):
    pipeline = make_pipeline(
        # punctuations are removed here, max df should make sure that no too common words are present
        TfidfVectorizer(encoding='utf-8', max_df=0.05), 
        LogisticRegression())
    pipeline.fit(X, Y)
    return pipeline

In [203]:
something1 = TfidfVectorizer(encoding='utf-8', max_df=0.05)
something2 = something1.fit_transform(X_gold_train['text'])#train_document_classifier2(X_gold_train['text'], Y_gold_train)
param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
search = GridSearchCV(LogisticRegression(), param_grid)
search.fit(something2, Y_gold_train)
print('Logistic Regression parameters: ', search.best_params_)

/home/koelbel/aml/my_project_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/koelbel/aml/my_project_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Logistic Regression parameters:  {'C': 1, 'penalty': 'l2'}


In [197]:
import seaborn as sns
C_Matrix = confusion_matrix(Ytest, Yguess, normalize="true")

labels = ["Antivaccine", "Provaccine"]
plt.figure(figsize=(18, 6))
seaborn.heatmap(C_Matrix, annot=True, square=True, cmap="Reds", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted label")
plt.ylabel("Actual label")
plt.title("Confusion matrix")
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [201]:
#vec = TfidfVectorizer()
clf = train_document_classifier(X_cs_train['text'], Y_cs_train)
clf2 = train_document_classifier2(X_gold_train['text'], Y_gold_train)
len(clf[0].stop_words_)
#clf[1].

#accuracy_score(Y_test, clf.predict(X_test['text']))
accuracy_score(Y_test, clf2.predict(X_test['text']))

0.7119840695148443